In [ ]:
from ultralytics import YOLO
from PIL import Image, ImageDraw
from roboflow import Roboflow
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import tensorflow as tf

In [183]:
model = YOLO('./runs/segment/train6/weights/best.pt')

In [ ]:
def average_pixels_in_block_for_curb(masks, image_size):
    tresholds = 5
    treshold_range = image_size[0] / tresholds
    average_x_values_curb_1 = []
    average_x_values_curb_2 = []
    average_x_values = []
    values = []
    for mask in masks:
        values.append(mask)
        
    mask_index = 0    
    for mask in values:
        mask = mask[np.argsort(mask[:, 1])]
        index = 0
        current_treshold_range = treshold_range
        points = 0
        
        for i in range(tresholds):
            sum = 0
            points = 0
            while mask[index][1] <= current_treshold_range:
                sum += mask[index][0]
                index+=1
                points+=1
                print(points)
                if index == mask.shape[0]:
                    index = 0
                    break
            if mask_index == 0:
                average_x_values_curb_1.append(sum / int(points))
                current_treshold_range += treshold_range
            if mask_index == 1:
                average_x_values_curb_2.append(sum / int(points))
                current_treshold_range += treshold_range
        index = 0
        mask_index += 1
        
    for i in range(tresholds):
        average_x_values.append((average_x_values_curb_1[i] + average_x_values_curb_2[i]) / 2)
         
    return average_x_values

In [ ]:
def average_pixels_in_block(masks, image_size):
    tresholds = 5
    treshold_range = image_size[0] / tresholds
    average_x_values = [image_size[1] / 2, image_size[1] / 2, image_size[1] / 2, image_size[1] / 2, image_size[1] / 2]
    values = []
    for mask in masks:
        values.append(mask)
        
    for mask in values:
        mask = mask[np.argsort(mask[:, 1])]
        index = 0
        current_treshold_range = treshold_range
        
        for i in range(tresholds):
            sum = 0
            while mask[index][1] <= current_treshold_range:
                sum += mask[index][0]
                index+=1
                if index == mask.shape[0]:
                    index = 0
                    break
            average_x_values[i] = (average_x_values[i] + (sum / int(treshold_range))) / 2
            current_treshold_range += treshold_range
        index = 0
         
    return average_x_values

In [207]:
result = model("Screenshot_2.png")
uniqueFromArray = []

def average_pixels_in_block(masks, image_size):
    tresholds = 4
    treshold_range = image_size[0] / tresholds
    average_x_values = [image_size[1] / 2, image_size[1] / 2, image_size[1] / 2, image_size[1] / 2, image_size[1] / 2]
    values = []
    for mask in masks:
        values.append(mask)
        
    for mask in values:
        mask = mask[np.argsort(mask[:, 1])]
        index = 0
        current_treshold_range = treshold_range
        
        for i in range(tresholds):
            sum = 0
            left_points = 0
            total_points = 0
            while mask[index][1] <= current_treshold_range:
                sum += mask[index][0]
                total_points += 1
                if mask[index][0] <= image_size[1] / 2:
                    left_points += 1
                index+=1
                if index == mask.shape[0]:
                    index = 0
                    break
                
            r_ratio = 0
            if left_points != 0:
                r_ratio = left_points / (total_points / 2)

            average_x_values[i] = ((average_x_values[i] + (sum / int(total_points))) / 2)
            current_treshold_range += treshold_range
            left_points = 0
        index = 0
         
    return average_x_values
    

for r in result:
    print(r.names)
    print(r.boxes.cls)
    print(r.orig_shape)
    
    # get the classes and setup empty mask array
    classes = r.boxes.cls.cpu().numpy()
    masks = []
    
    # extract the needed masks
    index = 0
    for mask in r.masks.xy:
        if classes[index] == 5:
            masks.append(mask)
        index+=1    
    
    result = average_pixels_in_block(masks, r.orig_shape)
    
    temp_mask = masks[0]
    black_image = np.zeros(r.orig_shape)
    black_image = Image.fromarray(black_image)
    temp_mask = temp_mask
    for point in temp_mask:
        black_image.putpixel((math.floor(point[0])-1, math.floor(point[1])-1), 255)
    draw = ImageDraw.Draw(black_image)
    #draw.polygon(temp_mask, fill="black",width=12, outline='blue')
    
    treshold = r.orig_shape[0] / 4
    y = treshold / 2
    for p in result:    
        draw.ellipse((p-10, y-10, p+10, y+10), fill='red', outline='green', width=5)
        y += treshold
    
    black_image.show()

image 1/1 c:\Users\damem\Desktop\Object-detection-with-YOLO\Screenshot_2.png: 512x640 1 Curb, 4 Grounds, 1 Sidewalk, 31.9ms
Speed: 8.0ms preprocess, 31.9ms inference, 8.0ms postprocess per image at shape (1, 3, 512, 640)


{0: 'Curb', 1: 'Floor', 2: 'Ground', 3: 'Hole', 4: 'Road', 5: 'Sidewalk', 6: 'Stairs'}
tensor([2., 2., 5., 0., 2., 2.], device='cuda:0')
(642, 817)
0
1.424390243902439
1.3
0.4
